In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import gpytorch
from gpytorch import means, kernels, likelihoods, distributions, lazy
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import xarray as xr
import tqdm
import utils as utils


base_dir = os.path.join(os.getcwd(), '..')
sys.path.append(base_dir)

from src.fair import run, get_params
from src.fair.ancil import get_gas_params, get_thermal_params
from src.preprocessing import load_emissions_dataset, load_response_dataset
from src.structures import Scenario, ScenarioDataset
import utils_spatial as spatial

<IPython.core.display.Javascript object>

In [25]:
train_keys = ['historical', 'ssp126', 'ssp370', 'ssp585']
test_keys = ['ssp245']
keys = train_keys + test_keys
inputs = {key: load_emissions_dataset(f'../data/inputs_{key}.nc') for key in keys}
outputs = {key: load_response_dataset(f'../data/outputs_{key}.nc') for key in keys}

In [26]:
def make_scenario(name, hist_scenario=None):
    time, _, emission, tas = utils.extract_arrays(inputs[name], outputs[name])
    scenario = Scenario(name=name,
                        timesteps=torch.from_numpy(time).float(),
                        emissions=torch.from_numpy(emission).float().T,
                        tas=torch.from_numpy(tas).float(),
                        hist_scenario=hist_scenario)
    return scenario

hist_scenario = make_scenario('historical')
ssps = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
scenarios = {'historical': hist_scenario}
for name in ssps:
    scenario = make_scenario(name, hist_scenario)
    scenarios[name] = scenario
    
train_scenarios = ScenarioDataset(scenarios=list([scenarios[key] for key in train_keys]),
                                  hist_scenario=hist_scenario)
test_scenarios = ScenarioDataset(scenarios=list([scenarios[key] for key in test_keys]),
                                 hist_scenario=hist_scenario)

In [27]:
spatial_hist_scenario = spatial.make_scenario(inputs, outputs, 'historical')
ssps = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
spatial_scenarios = {'historical': spatial_hist_scenario}
for name in ssps:
    scenario = spatial.make_scenario(inputs, outputs, name, spatial_hist_scenario)
    spatial_scenarios[name] = scenario

In [31]:
spatial_train_scenarios = ScenarioDataset(scenarios=list([spatial_scenarios[key] for key in train_keys]),
                                          hist_scenario=spatial_hist_scenario)
spatial_test_scenarios = ScenarioDataset(scenarios=list([spatial_scenarios[key] for key in test_keys]),
                                         hist_scenario=spatial_hist_scenario)

In [33]:
base_kwargs = get_params()

d = base_kwargs['d']
q = base_kwargs['q']

d_map = np.tile(d, 10 * 10).reshape(10 * 10, 3)
q_map = np.tile(q, 10 * 10).reshape(10 * 10, 3)

In [46]:
means_simple = spatial.compute_means(test_scenarios, d.reshape(1, 3), q.reshape(1, 3))
means_multiple = spatial.compute_means(test_scenarios, d_map, q_map)

In [57]:
foo = means_multiple[test_scenarios[0]].reshape(10, 10, 86, 3).permute(2, 0, 1, 3)

In [56]:
foo[:, 0, 0]

tensor([0.0007, 0.0023, 0.0043, 0.0064, 0.0088, 0.0117, 0.0149, 0.0182, 0.0217,
        0.0252, 0.0287, 0.0322, 0.0358, 0.0394, 0.0431, 0.0468, 0.0506, 0.0545,
        0.0584, 0.0624, 0.0665, 0.0705, 0.0746, 0.0779, 0.0799, 0.0811, 0.0819,
        0.0824, 0.0828, 0.0831, 0.0834, 0.0836, 0.0837, 0.0839, 0.0841, 0.0843,
        0.0844, 0.0846, 0.0848, 0.0849, 0.0851, 0.0853, 0.0854, 0.0856, 0.0857,
        0.0859, 0.0861, 0.0862, 0.0864, 0.0865, 0.0867, 0.0868, 0.0870, 0.0871,
        0.0873, 0.0874, 0.0876, 0.0877, 0.0879, 0.0880, 0.0882, 0.0883, 0.0885,
        0.0886, 0.0888, 0.0889, 0.0891, 0.0892, 0.0893, 0.0895, 0.0896, 0.0898,
        0.0899, 0.0901, 0.0902, 0.0903, 0.0905, 0.0906, 0.0907, 0.0909, 0.0910,
        0.0912, 0.0913, 0.0914, 0.0916, 0.0917, 0.0918, 0.0920, 0.0921, 0.0922,
        0.0923, 0.0925, 0.0926, 0.0927, 0.0929, 0.0930, 0.0931, 0.0932, 0.0934,
        0.0935, 0.0936, 0.0937, 0.0939, 0.0940, 0.0941, 0.0942, 0.0944, 0.0945,
        0.0946, 0.0947, 0.0948, 0.0950, 

In [23]:
d_map.reshape(96, 144, -1).shape

(96, 144, 3)

In [24]:
6842880 / 3

2280960.0

In [28]:
165 * 96 * 144 * 3

6842880

In [27]:
len(hist_scenario)

165

What do I need to do to run a simple experiment:
- Take the usual GP model
- ~Can I use ScenarioDataset? PRobably not, that's where I need to start... because this is going to be useful either way~
- Allow d and q to be functions
- Ugh, is it going to be compatible with all the subfunctions? Probably not, gonna have to be careful with that
- ~Can evaluate d, q than flatten into 3 * n_pixel boxes and then reshape. Good trick to avoid coding too much in case it fails (or even as a first attempt)~ -> cannot use the code as is because it won't scale... maybe only some latitudes/longitudes at first